<a href="https://colab.research.google.com/github/dagnybrand/AI-news-bias/blob/main/train_on_all_inputs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os.path
import json
import pandas as pd

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import SGDClassifier
import sklearn.model_selection

import tensorflow as tf
import tensorflow_hub as hub
import numpy as np

In [ ]:
if not os.path.exists('bias_data_short.json'):
  !wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1rF2ZPWDw2_N4E6Uo5SGG8W6YazuTDXnz' -O bias_data_short.json

In [ ]:
from pandas.core.arrays.categorical import NoNewAttributesMixin
from contextlib import nullcontext
with open('bias_data_short.json') as f:
  full_data_dict = json.loads(f.read())

headers = ['topic', 'source', 'bias', 'title', 'content']
df = pd.DataFrame()

topics = []
source = []
bias = []
title = []
content = []

lists = [topics, source, bias, title, content]

for item in full_data_dict:
  for header, data in zip(headers, lists):
    if item[header] is not None:
      data.append(item[header])
    else:
      data.append('null')

for header, data in zip(headers, lists):
  df[header] = data

y_df = df['bias'].copy()

x_df = df.drop(labels='bias', axis=1)

In [ ]:
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(x_df['title'])

print(X_train_counts)
X_train_counts.shape

In [ ]:
module_url = "https://tfhub.dev/google/universal-sentence-encoder/4"
model = hub.load(module_url)
print ("module %s loaded" % module_url)

module https://tfhub.dev/google/universal-sentence-encoder/4 loaded


In [ ]:
topic_embeddings = model(x_df['topic'])
content_embeddings = model(x_df['content'])
source_embeddings = model(x_df['source'])
title_embeddings = model(x_df['title'])

In [ ]:
em_df = pd.DataFrame()
em_df['topic'] = np.array(topic_embeddings).tolist()
em_df['content'] = np.array(content_embeddings).tolist()
em_df['title'] = np.array(title_embeddings).tolist()
em_df['source'] = np.array(source_embeddings).tolist()

In [ ]:
print(em_df)

                                                  topic  \
0     [-0.0628657415509224, -0.04393262043595314, -0...   
1     [-0.037389691919088364, -0.0498385950922966, 0...   
2     [0.017720667645335197, -0.015966027975082397, ...   
3     [-0.0628657415509224, -0.04393262043595314, -0...   
4     [-0.0628657415509224, -0.04393262043595314, -0...   
...                                                 ...   
996   [-0.061572831124067307, -0.006184494588524103,...   
997   [-0.025700176134705544, -0.038757916539907455,...   
998   [0.0036468892358243465, 0.033641085028648376, ...   
999   [-0.028081052005290985, -0.005823448300361633,...   
1000  [0.06156580522656441, -0.033410198986530304, 0...   

                                                content  \
0     [0.005978220608085394, -0.05688862502574921, -...   
1     [-0.022267526015639305, -0.05396590754389763, ...   
2     [-0.023165499791502953, -0.05102813243865967, ...   
3     [-0.032482728362083435, -0.05443748086690903, ...

In [109]:
em_df_exp = pd.DataFrame()

count = 0

# expand data frame so each coefficient of input vector for each category has a column
for col in em_df.columns:
  data = {}
  for item in em_df[col]:
    count = 0
    for element in item:
      if count in data:
        data[count].append(element)
      else:
        data[count] = [element]
      count += 1

  for key in data:
    em_df_exp[f'{col}{key}'] = data[key].copy()



<ipython-input-109-5c9ef982a1a7>:16: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  em_df_exp[f'{col}{key}'] = data[key].copy()
<ipython-input-109-5c9ef982a1a7>:16: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  em_df_exp[f'{col}{key}'] = data[key].copy()
<ipython-input-109-5c9ef982a1a7>:16: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented fra

In [110]:
print(em_df_exp)

        topic0    topic1    topic2    topic3    topic4    topic5    topic6  \
0    -0.062866 -0.043933 -0.013300 -0.017332 -0.007364  0.010291  0.014267   
1    -0.037390 -0.049839  0.055891  0.020852 -0.060803 -0.034120  0.023585   
2     0.017721 -0.015966  0.012558  0.053298  0.013441 -0.015035 -0.030523   
3    -0.062866 -0.043933 -0.013300 -0.017332 -0.007364  0.010291  0.014267   
4    -0.062866 -0.043933 -0.013300 -0.017332 -0.007364  0.010291  0.014267   
...        ...       ...       ...       ...       ...       ...       ...   
996  -0.061573 -0.006184 -0.028643  0.067156 -0.079502  0.026094 -0.010935   
997  -0.025700 -0.038758  0.075657  0.037729 -0.036311  0.027566  0.062982   
998   0.003647  0.033641  0.087970 -0.000100 -0.017960  0.041255 -0.026474   
999  -0.028081 -0.005823  0.075380  0.014799 -0.039165  0.029606 -0.022766   
1000  0.061566 -0.033410  0.062320  0.083477 -0.003510  0.020073  0.051909   

        topic7    topic8    topic9  ...  source502  source503  

In [111]:
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(em_df_exp, y_df, test_size=0.3, random_state=42)

In [113]:
sgd = SGDClassifier()

sgd.fit(X_train, y_train)

score = sgd.score(X_test, y_test)
print(score)

0.9302325581395349
